# Colloidspy Example Workflow

### Imports:

In [5]:
from colloidspy import cspy
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Step 1 - Load the images

CspyStack is python class that holds all of the images.

Each new processed version of the images we create will be held in attributes of the stack. Each original image can be found by indexing the stack:
- stack`[0]` would return the first image.

In [ ]:

directory = 'replace with the directory that the images are in'
stack = cspy.CspyStack(*cspy.load(os.path.join(director, '*.tiff')))

### Step 2 (optional) - Crop all images to the desired Region of Interest (ROI)

This brings up the first image. Click and drag from the top left to the bottom right to select the ROI.

Hit any key to continue. If it does not continue, hit 'q' to deactivate the selector, then hit any key.

Once cropped, ROI's can be found by its attribute:
- stack.cropped

A single cropped image (and likewise for all remaining attributes) can be accessed by indexing the attribute:
- stack.cropped`[0]` returns the first cropped image

In [ ]:
import matplotlib # only required if using a jupyter notebook
matplotlib.use('Qt5Agg')  # only required if using a jupyter notebook

stack.add_cropped()

### Step 3 - Binarize the images

Binarization enables the program to recognize what is a particle and what is background. Each pixel is either white (particle) or black (background).
There are three methods available, you should test them each and see which method works best for your images.

- Otsu threshold sets a cutoff pixel value, anything above is a "particle" anything below is "background". This method works well for most confocal images with even lighting and not too high particle density.
- Local threshold evaluates each pixel based on the surrounding pixels. This method is more flexible and works in most cases where the otsu method fails, but is much slower.
- Hysteresis threshold finds bright spots and fills in particles from each of those spots like nucleation points. This method may work for systems where local and otsu thresholds are not working well (rare).

Once the optimal method and their corresponding parameters have been chosen, it is best to only use one of the three methods for the final analysis to minimize memory usage.

Once once of these methods has been called, you can find the binarized stack at its respective attribute:

- stack.binary_otsu
- stack.binary_loc
- stack.binary_hyst

In [ ]:
stack.add_otsu()

In [ ]:
# 'block_size' sets how large the region is that is used to evaluate if a pixel should be white or black. Must be an odd number.
# Too large of a block_size can miss important features, too small can introduce noise, false white spots in background spaces, or false black spots in particle.
# Offset subtracts from the weighted mean of the of the region before thresholding
# Cutoff sets a minimum pixel value that can be considered a particle. Helpful if false white regions are showing up in the background.
stack.add_local_threshold(block_size=151, offset=1, cutoff=0)

In [ ]:
# All pixels with a value above 'high' are considered part of a particle.
# All pixels with a value below 'low' are considered background.
# Intermediate value pixels are evaluated based on whether or not they touch a pixel that has already been designated as a particle.
# This method can be fairly sensitive to the low and high pixel cutoffs set, so it may require experimentation.
stack.add_hysteresis_threshold(low=20, high=150)

### Step 4 - Remove noise

This cleans up pixel noise, leaving the major structuring elements of the binary images.
Pass in the binary stack you created.

Future analysis will be done on this stack.

In [ ]:
stack.add_cleaned(stack.binary_otsu)


### Step 5 (optional) - Detect particles + quality check

This is helpful when setting up the analysis to make sure everything is working correctly and accurately.
If this is skipped, particles will be detected in step 6.

In [ ]:
# min_distance is the minimum distance of particle centers (in pixels)
stack.find_particles(stack.cleaned, min_distance=3)

# see particles in the first image outlined in red
plt.imshow(cspy.view_particles(stack.cropped[0], stack.particles[0], weight=1))

### Step 6 - Analyze the stack

This function will do two things:
- Populate the attribute stack.particle_data with pandas dataframes of particle data for each image
- Return a dataframe with summaries of the particle data from each image

Options:
- save_dfs: True will save the dataframes for each image as a csv in the directory provided
- save_ims: True will save a binary image of the particles to the directory provided
- save_dir: directory the save images and dataframes (required if saving dfs or ims)
- im_titles: list of titles that you want each processed image and its dataframe to be saved as. If none is given, they will be numbered sequentially.
- imtype: image file type to save the binary images as

In [ ]:
overalldf = stack.analyze_stack(save_dfs=False, save_ims=False, save_dir=None, im_titles=None, imtype='tiff', min_distance=7)

## Helpful Information

cspy.load() will currently only work with greyscale, rgb, and rgba images. All images will be converted back to 8bit greyscale (0-255)

cspy.view_particles(image, particles, weight) will draw the particles over the image given, can be the binary version or the greyscale version. The image provided MUST be the same size as the particles. So, if you cropped the images, you can pass in either the binary image or the cropped image. If you did not crop the images, you can use the binary or the original image from the stack.

The attribute CspyStack.particles has each discrete particle tagged with an integer. So, all pixels of the first detected particle (top left) will have a value of '1', all pixels in the second particle will have a value of '2', etc. A pixel value of '0' indicates background.


### Extra methods

The rdf method will return the simple radial distribution function of the particles. This is based off of only two dimensions, and does not account for edge effects.

'image_index' is the index of the image from which you want to calculate the rdf. This returns a tuple of distance/avg particle radius and the rdf, and does not save it as an attribute. This method uses the particle centers from stack.particle_data found earlier in the analysis.

In [ ]:
distance, gr = stack.rdf(0)


The structure_factor method calculates the squared intensity of the fourier transform (2D FFT) of the image at the index given. It returns the structure factor over spatial frequency (in 1/px)

In [ ]:
sfactor = stack.structure_factor(0)

### List of all attributes:
- CspyStack.filenames - original filenames of each image imported
- CspyStack.cropped - image ROI's, set with add_cropped()
- CspyStack.crop_coords - (x,y) coordinates of image ROI
- CspyStack.binary_otsu - ROI's binarized with otsu threshold, set with add_otsu()
- CspyStack.binary_loc - ROI's binarized with local threshold, set with add_local_threshold()
- CspyStack.binary_hyst - ROI's binarized with hysteresis threshold, set with add_hysteresis_threshold()
- CspyStack.cleaned - Cleaned binary ROI's, set with add_cleaned(BinaryStack) where BinaryStack is the set of binary images chosen earlier with otsu, local, or hysteresis threshold methods
- CspyStack.particles - ROI where the pixels of all particles are assigned a unique integer labeled by particle
- CspyStack.particle_data - pandas dataframes of particle data in each image, held in a list

